## Example for SDK

This example shows how to use SDK to deploy a task (V1)

### Initialization

In [1]:
import sys
sys.path.insert(0, '..')

import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI , MCSAPI

# dev_url = "https://swanhub-cali.swanchain.io"
dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
# get user api_key in dashboard page: https://orchestrator-test.swanchain.io/provider-status
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)

api_key = os.getenv("MCS_API_KEY")
mcs_api = MCSAPI(api_key)

### Available hardware information

In [4]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
# hardwares_info

choose hardware config

In [5]:
device = 'C1ae.medium' #"G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

1
('C1ae.medium', 1, ['Quebec-CA'])


to simplify the process, here we use a existing `job_source_uri` which is a hello world application, used to create task.

In [6]:
job_source_uri = 'https://test-api.lagrangedao.org/spaces/5117e998-c623-4837-8af9-2b7b0ce2de7f'

### Contract and Payment

In [7]:
from swan import SwanContract
dotenv.load_dotenv("../.env")
 
pk = os.getenv('PK')
rpc = os.getenv('RPC')

c = SwanContract(pk, swan_api.contract_info)
 
# Test esimate lock revenue
r = c.estimate_payment(obj.id, 2)
print(r)

ContractLogicError: execution reverted

In [ ]:
r = c._approve_swan_token(2000000000000000000)
print(r)
 
r = c.lock_revenue('1', obj.id, 1)
print(r)

0x56984e04de56df85b51ea27a4c8498ccf4b266701ad2dfdea6615d7c6c31cd9c
0x6faf8e8c7e42aa22c52933ef44405f32e97ac4bfcf07455b5baeb6945cb29624


### Deploy task

This step shows how to use SDK's interface for deploying task, which calls Orchestrator's task deployment API (V1), given the `job_source_uri` and **payment information** (`paid`, `tx_hash`) in the previous steps.

In [ ]:
# Deploy task
result = swan_api.deploy_task(
            cfg_name=device, 
            region='Quebec-CA', 
            start_in=5, duration=3600*1, 
            job_source_uri=job_source_uri, 
            paid=1,
            tx_hash=r,
            wallet_address=os.getenv('WALLET'),
          )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

{'data': {'task': {'created_at': '1713294534', 'end_at': '1713298131', 'leading_job_id': None, 'refund_amount': None, 'status': 'created', 'task_detail_cid': 'https://plutotest.acl.swanipfs.com/ipfs/QmNRAovZvLZNyaGmH5imBaP9evLdT1N59AhbhmLKknVEJT', 'tx_hash': None, 'updated_at': '1713294534', 'uuid': '7bbb1e61-82a9-43e4-8337-786d194797c6'}}, 'message': 'Task_uuid created.', 'status': 'success'}
Task UUID: 7bbb1e61-82a9-43e4-8337-786d194797c6


The following step is optional, shows information when waiting for task being deployed.

In [ ]:
# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_uri = info['data']['jobs'][0]['job_result_uri']
        job_real_uri = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_uri)
        print("Job Real URL: ", job_real_uri)
        
        # break
        if status == 'deployToK8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(30)

buildImage
Job Result URL:  https://42f6d9f62851.acl.swanipfs.com/ipfs/QmQEW9Dyi9aEQYtVdMxA7qkr7gdiiNE9MJxNtjt9uSyUKc
Job Real URL:  https://cf24tc2oot.dev2.crosschain.computer
deployToK8s
Job Result URL:  https://42f6d9f62851.acl.swanipfs.com/ipfs/QmQEW9Dyi9aEQYtVdMxA7qkr7gdiiNE9MJxNtjt9uSyUKc
Job Real URL:  https://cf24tc2oot.dev2.crosschain.computer


### Show result

`job_real_uri` is for show the result of application you deployed.

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://cf24tc2oot.dev2.crosschain.computer']


In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


{
    "Hello": "World! Today - 06.21"
}
